<a href="https://colab.research.google.com/github/chmawnt/Complete-Feature-Selection/blob/master/Text_processing_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import tensorflow as tf
print(tf.__version__)

2.6.0


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df =  pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")


In [ ]:
print(len(train_df))
train_df = train_df.drop_duplicates('text', keep='last')
print(len(train_df))

In [ ]:
sns.countplot(train_df['target'])

In [ ]:
train_df['target'].value_counts()

In [ ]:
plt.figure(figsize=(15,40))
print(f'Unique keywords num={len(train_df.keyword.unique())}')
print(f'Unique keywords num={len(test_df.keyword.unique())}')
sns.countplot(y=train_df['keyword'], color=(0,0,1), label='Train')
sns.countplot(y=test_df['keyword'], color=(1,0,0), label='Test')
plt.legend()

In [ ]:
plt.figure(figsize=(15,100))
sns.countplot(data=train_df, y='keyword', hue='target')

In [ ]:
import re

def preprocess_text(text):
    text = re.sub(r"http\S+", "", text)
    return text

In [ ]:
train_df['text'] = train_df['text'].apply(preprocess_text)
test_df ['text'] = test_df['text'].apply(preprocess_text)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_df['text'].tolist(),\
                                                      train_df['target'].tolist(),\
                                                      test_size=0.01,\
                                                      stratify = train_df['target'].tolist(),\
                                                      random_state=0)


In [ ]:
batch_size = 32
seed = 42
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,y_valid)).batch(batch_size)
#print(train_ds)

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(8,activation ="relu",name ="hidden")(net)
    net = tf.keras.layers.Dense(1, activation= "sigmoid" , name='classifier')(net)
    return tf.keras.Model(text_input, net)
    
    
   classifier_model = build_classifier_model()

tf.keras.utils.plot_model(classifier_model)
for layer in classifier_model.layers[:-3]:
      layer.trainable = False

In [ ]:
classifier_model = build_classifier_model()

tf.keras.utils.plot_model(classifier_model)
for layer in classifier_model.layers[:-3]:
      layer.trainable = False

In [ ]:
classifier_model = build_classifier_model()

tf.keras.utils.plot_model(classifier_model)
for layer in classifier_model.layers[:-3]:
      layer.trainable = False

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs)

In [ ]:
probs = classifier_model.predict(test_df["text"]) 
threshold = 0.4
preds = np.where(probs[:,] > threshold, 1, 0)